In [34]:
import sys
sys.path.append('')    # ADD PATH TO REPOSITORY HERE SO YOU CAN CALL FUNCTIONS

import os

from pprint import pformat
from importlib import reload

import pandas as pd
from transformers import AutoTokenizer
import openai

import utils, openai_utils

In [52]:
reload(utils)
reload(openai_utils)

<module 'openai_utils' from '/home/igor/longevity/longevity/openai_utils.py'>

In [208]:
dataset_dir = './data/nlm/Transposable_element_landscape_in_Drosophila_populations_selected_for_longevity/'

In [209]:
os.listdir(dataset_dir)

['__MACOSX',
 'Average_Expression_per_Insertion.xlsx',
 '.ipynb_checkpoints',
 'description.txt',
 'outputs']

In [210]:
data_filename = 'Average_Expression_per_Insertion.xlsx'
header = 0

data_extension = data_filename.rsplit('.', 1)[-1]

description_fp = os.path.join(dataset_dir, 'description.txt')
if os.path.isfile(description_fp):
    with open(description_fp, 'r') as f:
        description = f.read()
else:
    description = ''

data_fp = os.path.join(dataset_dir, data_filename)

In [211]:
if data_extension in ['xls', 'xlsx']:
    print('\n'.join(pd.ExcelFile(data_fp).sheet_names))

Legend
Summary
Regime + Age (females)
Regime x Age (females)


In [212]:
sheet_name = 'Regime + Age (females)' # for xls

In [213]:
match data_extension:
    case 'csv':
        data = pd.read_csv(data_fp, header=header)
    case 'tsv' | 'tab':
        data = pd.read_csv(data_fp, sep='\t', header=header)
    case 'xls' | 'xlsx':
        if sheet_name:
            data = pd.read_excel(data_fp, sheet_name=sheet_name, header=header)
        else:
            data = pd.read_excel(data_fp, header=header)
    case _:
        print('Unexpected file extension.')

In [214]:
data

,GeneFBgn_or_TEfamily,Age_old_vs_young.baseMean,Age_old_vs_young.log2FoldChange,Age_old_vs_young.lfcSE,Age_old_vs_young.stat,Age_old_vs_young.pvalue,Age_old_vs_young.padj,Regime_Sel_vs_Cont.baseMean,Regime_Sel_vs_Cont.log2FoldChange,Regime_Sel_vs_Cont.lfcSE,Regime_Sel_vs_Cont.stat,Regime_Sel_vs_Cont.pvalue,Regime_Sel_vs_Cont.padj
0,1360,19.215358,-0.037839,0.172999,-0.218726,0.826864,0.919146,19.215358,-0.249780,0.173165,-1.442438,1.491789e-01,0.323107
1,412,34.934478,-0.482065,0.135795,-3.549934,0.000385,0.004163,34.934478,-0.718406,0.136388,-5.267379,1.383855e-07,0.000005
2,ACCORD,40.701922,0.267794,0.249971,1.071303,0.284033,0.526333,40.701922,0.078430,0.249958,0.313771,7.536947e-01,0.867889
3,AF222049,47.339745,-0.303948,0.238337,-1.275287,0.202208,0.432361,47.339745,-0.860454,0.238539,-3.607183,3.095394e-04,0.003459
4,AF541951,15.045754,-0.427709,0.238881,-1.790472,0.073378,0.224279,15.045754,-0.943033,0.241063,-3.911975,9.154443e-05,0.001297
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13380,FBgn0085819,4.081353,0.362049,0.412987,0.876658,0.380672,0.621989,4.081353,0.250045,0.412939,0.605525,5.448304e-01,0.726636
13381,FBgn0265592,0.614389,-0.596891,1.199598,-0.497575,0.618783,NaN,0.614389,1.010618,1.205353,0.838441,4.017830e-01,NaN
13382,FBgn0036953,6.031805,0.682737,0.497381,1.372665,0.169857,0.388206,6.031805,0.752479,0.498064,1.510808,1.308373e-01,0.297576
13383,FBgn0264350,0.053149,-0.242132,3.026188,-0.080012,0.936228,NaN,0.053149,0.231245,3.026188,0.076414,9.390894e-01,NaN


In [215]:
data.columns

Index(['GeneFBgn_or_TEfamily', 'Age_old_vs_young.baseMean',
       'Age_old_vs_young.log2FoldChange', 'Age_old_vs_young.lfcSE',
       'Age_old_vs_young.stat', 'Age_old_vs_young.pvalue',
       'Age_old_vs_young.padj', 'Regime_Sel_vs_Cont.baseMean',
       'Regime_Sel_vs_Cont.log2FoldChange', 'Regime_Sel_vs_Cont.lfcSE',
       'Regime_Sel_vs_Cont.stat', 'Regime_Sel_vs_Cont.pvalue',
       'Regime_Sel_vs_Cont.padj'],
      dtype='object')

In [182]:
data.drop('Unnamed: 7', axis='columns', inplace=True)

KeyError: "['Unnamed: 7'] not found in axis"

In [216]:
data_sample = data.sample(20)
data_str = utils.df_to_str(data_sample)

In [217]:
print(data_str)

| GeneFBgn_or_TEfamily   |   Age_old_vs_young.baseMean |   Age_old_vs_young.log2FoldChange |   Age_old_vs_young.lfcSE |   Age_old_vs_young.stat |   Age_old_vs_young.pvalue |   Age_old_vs_young.padj |   Regime_Sel_vs_Cont.baseMean |   Regime_Sel_vs_Cont.log2FoldChange |   Regime_Sel_vs_Cont.lfcSE |   Regime_Sel_vs_Cont.stat |   Regime_Sel_vs_Cont.pvalue |   Regime_Sel_vs_Cont.padj |
|------------------------|-----------------------------|-----------------------------------|--------------------------|-------------------------|---------------------------|-------------------------|-------------------------------|-------------------------------------|----------------------------|---------------------------|-----------------------------|---------------------------|
| FBgn0266683            |                   0.0523803 |                       -0.24213    |                3.02619   |              -0.0800117 |               0.936228    |            nan          |                     0.0523803 

In [218]:
messages = [
    {'role': 'user', 'content': llama.dataset_columns_description_prompt 
                                + llama.dataset_input.format(dataset_description=description, dataset=data_str)}
]

In [219]:
generation_params = {
    'temperature': 0.0,
    'max_tokens': 4096,
}

In [220]:
model_name = 'gpt-4'

In [221]:
response = openai_utils.openai_generate(messages=messages, **generation_params)
descriptions = utils.postprocess_response(response.choices[0].message.content)

In [222]:
descriptions

{'GeneFBgn_or_TEfamily': {'description': 'Unique identifier for the gene or transposable element (TE) family.',
  'data_type': 'string'},
 'Age_old_vs_young.baseMean': {'description': 'The mean base value when comparing old versus young age groups.',
  'data_type': 'float'},
 'Age_old_vs_young.log2FoldChange': {'description': 'The log2 fold change when comparing old versus young age groups.',
  'data_type': 'float'},
 'Age_old_vs_young.lfcSE': {'description': 'The standard error of the log2 fold change when comparing old versus young age groups.',
  'data_type': 'float'},
 'Age_old_vs_young.stat': {'description': 'The statistical value when comparing old versus young age groups.',
  'data_type': 'float'},
 'Age_old_vs_young.pvalue': {'description': 'The p-value when comparing old versus young age groups.',
  'data_type': 'float'},
 'Age_old_vs_young.padj': {'description': 'The adjusted p-value when comparing old versus young age groups.',
  'data_type': 'float'},
 'Regime_Sel_vs_Cont.b

In [223]:
delimeter = '='*100+'\n\n'

outputs = [
    'GENERATION PARAMETERS\n\n' + pformat(generation_params, sort_dicts=False)+'\n\n',
    'INPUT PROMPT\n\n' + messages[0]['content'],
    'GENERATED DESCRIPTIONS\n\n' + pformat(descriptions, sort_dicts=False, width=100)
]

output = delimeter.join(outputs)

data_name = data_filename.rsplit('.', 1)[0]
name_parts = [model_name, data_name, 'txt']
if sheet_name:
    name_parts.insert(-1, sheet_name)

output_filename = '.'.join(name_parts)

outputs_path = os.path.join(dataset_dir, 'outputs')
if not os.path.isdir(outputs_path):
    os.makedirs(outputs_path)
    
output_fp = os.path.join(outputs_path, output_filename)

with open(output_fp, 'w') as f:
    f.write(output)